In [1]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [35]:
import os
import json
import time
import pandas as pd
import requests

from langchain import hub
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import ChatUpstage
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv

In [11]:
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
UPSTAGE_API_KEY = os.environ.get('UPSTAGE_API_KEY')
LANGCHAIN_API_KEY = os.environ.get('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = 'EXP04_GC' # 프로젝트명 수정
LANGCHAIN_PROJECT = os.environ.get('LANGCHAIN_PROJECT')

print(f'LangSmith Project: {LANGCHAIN_PROJECT}')

LangSmith Project: EXP04_GC


In [18]:
load_dotenv()

True

In [7]:
# 데이터 구성
file_path = '../data/documents.jsonl'
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

loader = JSONLoader(
    file_path=file_path,
    jq_schema='.',
    text_content=False,
    json_lines=True,
)
temp = loader.load()

seq_num = 1
documents = []
for tmp in temp:
    data = json.loads(tmp.page_content)
    doc = Document(page_content=data['content'], metadata={
        'docid': data['docid'],
        'src': data['src'],
        'source': '/data/ephemeral/home/upstage-ai-final-ir2/upstage-ai-final-ir2/HM/data/documents.jsonl',
        'seq_num': seq_num,
    })
    documents.append(doc)
    seq_num += 1


In [8]:
# Split
splitter = CharacterTextSplitter(
    separator='',
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)
split_documents = splitter.split_documents(documents)

In [15]:
# Embedding
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY, 
    model="solar-embedding-1-large"
)

In [19]:
# 벡터 저장소 생성
# pip install faiss-cpu
folder_path = f'./faiss_{LANGCHAIN_PROJECT}'
if not os.path.exists(folder_path):
    print('Vector Store 생성 중')
    vectorstore = FAISS.from_documents(
        documents=split_documents,
        embedding=embeddings,
    )
    vectorstore.save_local(folder_path=folder_path)
    print('Vector Store 생성 및 로컬 저장 완료')
else:
    vectorstore = FAISS.load_local(
        folder_path=folder_path, 
        embeddings=embeddings, 
        allow_dangerous_deserialization=True
    )
    print('Vector Store 로컬에서 불러옴')


Vector Store 생성 중
Vector Store 생성 및 로컬 저장 완료


In [20]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
prompt = hub.pull("rlm/rag-prompt")


In [28]:
# LLM과 검색엔진을 활용한 RAG 구현 (기존 코드와 동일)
retriever = vectorstore.as_retriever(k=5)
chat = ChatUpstage(model='solar-1-mini-chat', temperature=0)

In [39]:
# Groundedness Check 함수 추가
def check_groundedness(context, response):
    url = "https://api.upstage.ai/v1/solar/chat/completions"
    headers = {
        "Authorization": f"Bearer {UPSTAGE_API_KEY}",
        "Content-Type": "application/json"
    }
    
    # data = {
    #     "model": "solar-1-mini-groundedness-check",
    #     "messages": [
    #         {"role": "user", "content": context},
    #         {"role": "assistant", "content": response}
    #     ],
    #     "temperature": 0
    # }
    
    data = {
        "model": "solar-1-mini-groundedness-check",
        "messages": [
            {"role": "user", "content": messages[0]["content"]},
            {"role": "assistant", "content": answer}
        ],
        "temperature": 0
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()['choices'][0]['message']['content']
    else:
        return "Error in groundedness check"

In [40]:
def format_docs(docs):
    global references
    references = docs
    return '\n\n'.join(doc.page_content for doc in docs)

In [41]:
def answer_question(messages):
    global references
    response = {"topk": "", "answer": "", "references": "", "groundedness": ""}

    rag_chain = (
        {'context': retriever | format_docs, 'question': RunnablePassthrough()}
        | prompt
        | chat
        | StrOutputParser()
    )

    history = '\n'.join([f"{message['role']}: {message['content']}" for message in messages]) + '\n'
    answer = rag_chain.invoke(history)

    ref_content = [reference.page_content for reference in references]
    topk = [reference.metadata['docid'] for reference in references]

    # Groundedness Check 수행
    context = '\n'.join(ref_content)
    groundedness = check_groundedness(context, answer)

    response["topk"] = topk
    response["answer"] = answer
    response["references"] = ref_content
    response["groundedness"] = groundedness

    return response

In [42]:
def print_non_grounded_results(output_filename):
    non_grounded = []
    with open(output_filename, 'r') as file:
        for line in file:
            result = json.loads(line)
            if result['groundedness'] != 'grounded':
                non_grounded.append(result)
    
    if non_grounded:
        print(f"\n총 {len(non_grounded)}개의 non-grounded 결과가 있습니다:\n")
        for item in non_grounded:
            print(f"Eval ID: {item['eval_id']}")
            print(f"Question: {item.get('question', 'N/A')}")  # 질문이 저장되어 있다면 출력
            print(f"Answer: {item['answer']}")
            print(f"References:")
            for ref in item['references']:
                print(f"  - {ref}")
            print(f"Groundedness: {item['groundedness']}")
            print("-" * 50)
    else:
        print("\n모든 결과가 grounded입니다.")

In [43]:
# 평가를 위한 파일을 읽어서 각 평가 데이터에 대해서 결과 추출후 파일에 저장
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as eval_lines, open(output_filename, 'w') as output_lines:
        idx = 0
        for eval_line in eval_lines:
            j = json.loads(eval_line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = answer_question(j["msg"])
            print(f'Answer: {response["answer"]}')
            print(f'Groundedness: {response["groundedness"]}\n')

            output = {
                "eval_id": j["eval_id"], 
                "question": j["msg"][-1]["content"],  # 질문 저장
                "topk": response["topk"], 
                "answer": response["answer"], 
                "references": response["references"],
                "groundedness": response["groundedness"]
            }
            output_lines.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1

In [44]:
# 평가 실행
eval_rag('../data/eval.jsonl', '../output/EXP04_GC.csv')

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]


NameError: name 'messages' is not defined

In [ ]:
# 평가 결과 중 non-grounded 항목 출력
print_non_grounded_results('../output/EXP04_GC.csv')